In [4]:
import pandas as pd
import json 

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

import numpy as np
import scipy
from scipy import stats
from scipy import optimize

bokeh.io.output_notebook()

Loading BokehJS ...

In [6]:
fname = '../CAPA4_ATII_quant.csv'

# Load in the data
df = pd.read_csv(fname, comment = '#')

df.dropna()

,Virus,Animal,Section,Replicate,Positive,ATII
0,AAV5,1,1,1,1.0,1.0
1,AAV5,1,1,2,3.0,0.0
2,AAV5,1,1,3,0.0,0.0
3,AAV5,1,1,4,0.0,0.0
4,AAV5,1,2,1,0.0,0.0
...,...,...,...,...,...,...
125,AAV-CAP.A4,4,2,2,41.0,19.0
126,AAV-CAP.A4,4,2,3,48.0,10.0
127,AAV-CAP.A4,4,2,4,38.0,10.0
128,AAV-CAP.A4,5,1,1,122.0,18.0


In [7]:
grouped = df.groupby(['Virus','Animal'])

df_positive = grouped['Positive'].sum().reset_index()
df_positive['ATII'] = grouped['ATII'].sum().reset_index()['ATII']

df_positive['Fraction ATII Positive'] = df_positive['ATII']/df_positive['Positive']

df_positive = df_positive.dropna()

df_positive

,Virus,Animal,Positive,ATII,Fraction ATII Positive
0,AAV-CAP.A4,1,540.0,256.0,0.474074
1,AAV-CAP.A4,2,499.0,182.0,0.364729
2,AAV-CAP.A4,3,438.0,205.0,0.468037
3,AAV-CAP.A4,4,392.0,145.0,0.369898
4,AAV-CAP.A4,5,314.0,46.0,0.146497
6,AAV5,1,8.0,1.0,0.125000
7,AAV5,2,6.0,0.0,0.000000
8,AAV5,3,6.0,3.0,0.500000
9,AAV5,4,7.0,0.0,0.000000
10,AAV5,5,3.0,1.0,0.333333


In [8]:
grouped = df_positive.groupby(['Virus', 'Animal'])
df_positive_grouped = grouped['Fraction ATII Positive'].mean().reset_index()

In [9]:
# Store the data for plotting
source = ColumnDataSource(df_positive_grouped)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Fraction of GFP+ co-localized with proSPC expression')

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.2, range=p.x_range), y='Fraction ATII Positive', source=source, alpha=1)

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

# Code to add mean and standard deviation lines

# i = 0
# for virus in ['AAV-CAP.A4', 'AAV5', 'AAV9']:
#     inds = df_positive_grouped['Virus'] == virus
    
#     p.hbar(np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
#     p.hbar(np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']) + np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
#     p.hbar(np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']) - np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
#     p.segment(bar_list[i], np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), bar_list[i], np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']) + np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']) , color = 'black')
#     p.segment(bar_list[i], np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), bar_list[i], np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']) - np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']), color = 'black')
    
#     i += 1

# Simply modify plot characteristics 
p.plot_height = 350
p.plot_width = 350

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

p.background_fill_color = None 
p.border_fill_color = None

p.output_backend = "svg"

show(p)

In [10]:
grouped = df.groupby(['Virus','Animal'])

df_ATII = grouped['ATII'].mean().reset_index()

df_ATII = df_ATII.dropna()

In [11]:
inds_1 = df_ATII['Virus'] == 'AAV-CAP.A4'
inds_2 = df_ATII['Virus'] == 'AAV9'
inds_3 = df_ATII['Virus'] == 'AAV5'

np.mean(df_ATII.loc[inds_1, 'ATII']) / np.mean(df_ATII.loc[inds_2, 'ATII']), np.mean(df_ATII.loc[inds_1, 'ATII']) / np.mean(df_ATII.loc[inds_3, 'ATII'])

(33.32571428571429, 116.64)

In [12]:
inds = df_ATII['Virus'] == 'AAV9'

df_ATII['Normalized Transduction'] = df_ATII['ATII'] / np.mean(df_ATII.loc[inds, 'ATII'])

In [13]:
grouped = df_ATII.groupby(['Virus', 'Animal'])
df_ATII_grouped = grouped['Normalized Transduction'].mean().reset_index()

In [14]:
# Store the data for plotting
source = ColumnDataSource(df_ATII_grouped)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'GFP+ / ATII+ normalized to AAV9')

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.2, range=p.x_range), y='Normalized Transduction', source=source, alpha=1)

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

# Code to add mean and standard deviation lines

# i = 0
# for virus in ['AAV-CAP.A4', 'AAV5', 'AAV9']:
#     inds = df_positive_grouped['Virus'] == virus
    
#     p.hbar(np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
#     p.hbar(np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']) + np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
#     p.hbar(np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']) - np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
#     p.segment(bar_list[i], np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), bar_list[i], np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']) + np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']) , color = 'black')
#     p.segment(bar_list[i], np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']), bar_list[i], np.mean(df_positive_grouped.loc[inds,'Fraction ATII Positive']) - np.std(df_positive_grouped.loc[inds,'Fraction ATII Positive']), color = 'black')
    
#     i += 1

# Simply modify plot characteristics 
p.plot_height = 350
p.plot_width = 350

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

p.background_fill_color = None 
p.border_fill_color = None

p.output_backend = "svg"

show(p)

In [16]:
grouped = df_ATII_grouped.groupby(['Virus'])

df_ATII_grouped = grouped['Normalized Transduction'].mean().reset_index()

df_ATII_grouped = df_ATII_grouped.dropna()

In [17]:
df_ATII_grouped

,Virus,Normalized Transduction
0,AAV-CAP.A4,33.325714
1,AAV5,0.285714
2,AAV9,1.000000
